In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score


df1 = pd.read_csv('cleaned_data.dat', sep=' ')                                                         
df2 = pd.read_csv('noise_data.dat', sep=' ')
df1

,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
0,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
1,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
2,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
3,107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,bus
4,97,43,73,173,65,6,153,42,19,143,176,361,172,66,13,1,200,204,bus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,93,39,87,183,64,8,169,40,20,134,200,422,149,72,7,25,188,195,saab
809,89,46,84,163,66,11,159,43,20,159,173,368,176,72,1,20,186,197,van
810,106,54,101,222,67,12,222,30,25,173,228,721,200,70,3,4,187,201,saab
811,86,36,78,146,58,7,135,50,18,124,155,270,148,66,0,25,190,195,saab


In [27]:
x1 = df1.drop('van', axis=1)

x2 = df1.drop(['95', '48', '83', '178', 'van'], axis=1)
x1.shape, x2.shape

# Load the data into a DataFrame
df = pd.read_csv('cleaned_data.dat', header=None, sep=' ')

# Drop the extra column
# df = df.drop(columns=[19])

# Rename the last column as the target variable
df.rename(columns={18: 'target'}, inplace=True)

# Convert the target variable to string datatype
df['target'] = df['target'].astype(str)

# Scale the integer features
scaler = StandardScaler()
X = np.abs(scaler.fit_transform(df.iloc[:, :-1]))
X = df.iloc[:, :-1]

# Perform feature selection
selector = SelectKBest(chi2, k=10)
X_new = selector.fit_transform(X, df['target'])

# Perform dimensionality reduction
pca = PCA(n_components=10)
X_new = pca.fit_transform(X_new)

# Handle class imbalance
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_new, df['target'])
X_resampled.shape
X_resampled
x1 = X_resampled
y1 = y_resampled
x1.shape, y1.shape

((836, 10), (836,))

In [28]:
# y1 = df1['van'].copy()
# y1.shape

In [29]:
# from sklearn.feature_selection import VarianceThreshold

# selection = VarianceThreshold((.8 * (1 - .8)))
# x1 = selection.fit_transform(x1)

# selection = VarianceThreshold((.8 * (1 - .8)))
# x2 = selection.fit_transform(x2)
# x1.shape, x2.shape

In [30]:
# spliting the training set and testing set
from sklearn.model_selection import train_test_split

x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.20, random_state=100)

x1_train.shape, x1_test.shape

((668, 10), (168, 10))

In [31]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score

K nearest neighbors

In [25]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(5) # Define classifier
knn.fit(x1_train, y1_train) # Train model

# Make predictions
y_train_pred = knn.predict(x1_train)
y_test_pred = knn.predict(x1_test)

# Training set performance
knn_train_accuracy = accuracy_score(y1_train, y_train_pred) # Calculate Accuracy
knn_train_mcc = matthews_corrcoef(y1_train, y_train_pred) # Calculate MCC
knn_train_f1 = f1_score(y1_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
knn_test_accuracy = accuracy_score(y1_test, y_test_pred) # Calculate Accuracy
knn_test_mcc = matthews_corrcoef(y1_test, y_test_pred) # Calculate MCC
knn_test_f1 = f1_score(y1_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % knn_train_accuracy)
print('- MCC: %s' % knn_train_mcc)
print('- F1 score: %s' % knn_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % knn_test_accuracy)
print('- MCC: %s' % knn_test_mcc)
print('- F1 score: %s' % knn_test_f1)

Model performance for Training set
- Accuracy: 0.7574850299401198
- MCC: 0.6778148428259196
- F1 score: 0.750506019015683
----------------------------------
Model performance for Test set
- Accuracy: 0.6071428571428571
- MCC: 0.48159099799388627
- F1 score: 0.5855270865700533


C:\Users\james\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\james\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

Support vector machine (Radial basis function kernel)

In [ ]:
from sklearn.svm import SVC

svm_rbf = SVC(gamma=2, C=1)
svm_rbf.fit(x1_train, y1_train)

# Make predictions
y_train_pred = svm_rbf.predict(x1_train)
y_test_pred = svm_rbf.predict(x1_test)

# Training set performance
svm_rbf_train_accuracy = accuracy_score(y1_train, y_train_pred) # Calculate Accuracy
svm_rbf_train_mcc = matthews_corrcoef(y1_train, y_train_pred) # Calculate MCC
svm_rbf_train_f1 = f1_score(y1_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
svm_rbf_test_accuracy = accuracy_score(y1_test, y_test_pred) # Calculate Accuracy
svm_rbf_test_mcc = matthews_corrcoef(y1_test, y_test_pred) # Calculate MCC
svm_rbf_test_f1 = f1_score(y1_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % svm_rbf_train_accuracy)
print('- MCC: %s' % svm_rbf_train_mcc)
print('- F1 score: %s' % svm_rbf_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % svm_rbf_test_accuracy)
print('- MCC: %s' % svm_rbf_test_mcc)
print('- F1 score: %s' % svm_rbf_test_f1)

Model performance for Training set
- Accuracy: 1.0
- MCC: 1.0
- F1 score: 1.0
----------------------------------
Model performance for Test set
- Accuracy: 0.19047619047619047
- MCC: 0.0
- F1 score: 0.06095238095238095


Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=5) # Define classifier
dt.fit(x1_train, y1_train) # Train model

# Make predictions
y_train_pred = dt.predict(x1_train)
y_test_pred = dt.predict(x1_test)

# Training set performance
dt_train_accuracy = accuracy_score(y1_train, y_train_pred) # Calculate Accuracy
dt_train_mcc = matthews_corrcoef(y1_train, y_train_pred) # Calculate MCC
dt_train_f1 = f1_score(y1_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
dt_test_accuracy = accuracy_score(y1_test, y_test_pred) # Calculate Accuracy
dt_test_mcc = matthews_corrcoef(y1_test, y_test_pred) # Calculate MCC
dt_test_f1 = f1_score(y1_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % dt_train_accuracy)
print('- MCC: %s' % dt_train_mcc)
print('- F1 score: %s' % dt_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % dt_test_accuracy)
print('- MCC: %s' % dt_test_mcc)
print('- F1 score: %s' % dt_test_f1)

Model performance for Training set
- Accuracy: 0.781437125748503
- MCC: 0.716053393362593
- F1 score: 0.7702978408571352
----------------------------------
Model performance for Test set
- Accuracy: 0.6190476190476191
- MCC: 0.5088361961701604
- F1 score: 0.5942216624231699


Neural Network

In [38]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=1, max_iter=1000)
mlp.fit(x1_train, y1_train)

# Make predictions
y_train_pred = mlp.predict(x1_train)
y_test_pred = mlp.predict(x1_test)

# Training set performance
mlp_train_accuracy = accuracy_score(y1_train, y_train_pred) # Calculate Accuracy
mlp_train_mcc = matthews_corrcoef(y1_train, y_train_pred) # Calculate MCC
mlp_train_f1 = f1_score(y1_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set performance
mlp_test_accuracy = accuracy_score(y1_test, y_test_pred) # Calculate Accuracy
mlp_test_mcc = matthews_corrcoef(y1_test, y_test_pred) # Calculate MCC
mlp_test_f1 = f1_score(y1_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % mlp_train_accuracy)
print('- MCC: %s' % mlp_train_mcc)
print('- F1 score: %s' % mlp_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % mlp_test_accuracy)
print('- MCC: %s' % mlp_test_mcc)
print('- F1 score: %s' % mlp_test_f1)

Model performance for Training set
- Accuracy: 0.7994011976047904
- MCC: 0.7451125741400471
- F1 score: 0.7850048553478955
----------------------------------
Model performance for Test set
- Accuracy: 0.6428571428571429
- MCC: 0.5508696804085369
- F1 score: 0.6205225700317935


Build Stacked Model

In [39]:
# Define estimators
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    ('knn',knn),
    ('svm_rbf',svm_rbf),
    ('dt',dt),
    ('mlp',mlp) ]

# Build stack model
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression(solver='lbfgs', max_iter=1000))

# Train stacked model
stack_model.fit(x1_train, y1_train)

# Make predictions
y_train_pred = stack_model.predict(x1_train)
y_test_pred = stack_model.predict(x1_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y1_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y1_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y1_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set model performance
stack_model_test_accuracy = accuracy_score(y1_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y1_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y1_test, y_test_pred, average='weighted') # Calculate F1-score

print('Stacking performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print('----------------------------------')
print('Stacking performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)

# ----------------------------------------------------------------------------------------------------

# Build voting mechanism
voting_clf = VotingClassifier(
    estimators= estimator_list, voting='hard')

# Train voting mechanism
voting_clf.fit(x1_train, y1_train)

# Make predictions
y_train_pred2 = voting_clf.predict(x1_train)
y_test_pred2 = voting_clf.predict(x1_test)

# Training set model performance
vote_model_train_accuracy = accuracy_score(y1_train, y_train_pred2)

# Test set model performance
vote_model_test_accuracy = accuracy_score(y1_test, y_test_pred2)

print('----------------------------------')
print('Voting performance for Training set')
print('- Accuracy: %s' % vote_model_train_accuracy)
print('----------------------------------')
print('Voting performance for Test set')
print('- Accuracy: %s' % vote_model_test_accuracy)

Stacking performance for Training set
- Accuracy: 0.812874251497006
- MCC: 0.7527997567991488
- F1 score: 0.8038853300926594
----------------------------------
Stacking performance for Test set
- Accuracy: 0.6845238095238095
- MCC: 0.5849381664591969
- F1 score: 0.675933763656978
----------------------------------
Voting performance for Training set
- Accuracy: 0.9206586826347305
----------------------------------
Voting performance for Test set
- Accuracy: 0.6547619047619048


C:\Users\james\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\james\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

In [73]:
# defining the classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

svm_clf = SVC(C=0.1)
nb_clf = MultinomialNB()
dt_clf = DecisionTreeClassifier()
mlp_clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [150]:
#defining the voting mechanism
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('svm', svm_clf), ('nb', nb_clf), ('df', dt_clf), ('mlp', mlp_clf)], voting='hard')

In [75]:
voting_clf.fit(x1_train, y1_train)

VotingClassifier(estimators=[('svm', SVC(C=0.1)), ('nb', MultinomialNB()),
                             ('df', DecisionTreeClassifier()),
                             ('mlp',
                              MLPClassifier(alpha=1e-05,
                                            hidden_layer_sizes=(5, 2),
                                            random_state=1, solver='lbfgs'))])

In [77]:
for clf in (svm_clf, nb_clf, dt_clf, mlp_clf):
    clf.fit(xs_train, ys_train)

In [78]:
# measure the train and test accuracy of each classifier, including the ensembled one
from sklearn.metrics import accuracy_score

for clf in (svm_clf, nb_clf, dt_clf, mlp_clf, voting_clf):
    print(clf.__class__.__name__)
    ys_pred = clf.predict(xs_train)
    print('\ttrain:', accuracy_score(ys_train, ys_pred))
    ys_pred = clf.predict(xs_test)
    print('\ttest:', accuracy_score(ys_test, ys_pred))

SVC
	train: 0.40307692307692305
	test: 0.4110429447852761
MultinomialNB
	train: 0.4369230769230769
	test: 0.4171779141104294
DecisionTreeClassifier
	train: 1.0
	test: 0.754601226993865
MLPClassifier
	train: 0.26615384615384613
	test: 0.2147239263803681
VotingClassifier
	train: 0.5507692307692308
	test: 0.4601226993865031
